In [ ]:
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import json
from gensim.models import Word2Vec

Vector_size = 256
Top_Hit = 50
Max_Demaged_Image = 5


def extract_text(text):
    parts = text.split(":")
    if len(parts) > 1:
        return parts[1].strip()
    else:
        return text.strip()
    

model = Word2Vec.load("word2vec.model")


meta_file_path = "Meta.txt"


word_vectors = {}


with open(meta_file_path, 'r', encoding='utf-8') as meta_file:
    lines = meta_file.readlines()

cnt = 0
for line in lines:
    extracted_text = extract_text(line)
    words_in_line = extracted_text.split()
    for word in words_in_line:
        if word in model.wv:
            vector = model.wv[word]
           
        else :
            vector = np.zeros(Vector_size,)
        word_vectors[cnt] = vector
        cnt += 1

print(cnt)


In [ ]:
np.array(list(word_vectors.values())).shape

In [ ]:
all_vectors = np.array(list(word_vectors.values())).reshape(cnt, Vector_size)

In [ ]:

meta_file_path = "Meta.txt"

word_vectors = {}

# Meta.txt 파일을 엽니다.
with open(meta_file_path, 'r', encoding='utf-8') as meta_file:
    lines = meta_file.readlines()


word_list=[]
for line in lines:
    extracted_text = extract_text(line)
    words_in_line = extracted_text.split()
    for word in words_in_line:
        word_list.append(word)


In [ ]:

model = Word2Vec.load("word2vec.model")

json_file_path = "roadsign_data_info_damaged_sample_0928_수정.json"


with open(json_file_path, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)


annotations = data.get("annotations", [])


bbox_texts_by_image_id = {}

#print(annotations)


for annotation in annotations:
    damaged_status = annotation.get("damaged")
    bbox_text = annotation.get("bbox_text")
    image_id = annotation.get("image_id")
    

    if image_id not in bbox_texts_by_image_id:
        bbox_texts_by_image_id[image_id] = {"N": [], "Y": []}
    
    if damaged_status == "N":
        bbox_texts_by_image_id[image_id]["N"].append(bbox_text)
    elif damaged_status == "Y":
        bbox_texts_by_image_id[image_id]["Y"].append(bbox_text)


res = []
bbox_N_idx = 0
try_count = 0
for image_id, bbox_texts in bbox_texts_by_image_id.items():
    
    print("Image ID:", image_id)
    
    res_image_id = []

    print("['damaged':'N'] Bbox Texts:")
    for bbox_text in bbox_texts["N"]:
        if bbox_text in model.wv:
            vector = model.wv[bbox_text]
            reshape_vector = vector.reshape(1, Vector_size)
            res_image_id.append(cosine_similarity(reshape_vector,all_vectors))
            # print(res)
            # print(f"{bbox_text}: {vector}")
        
        else:
            print(f"{bbox_text} is not in the vocabulary.")
            res_image_id.append(np.zeros(cnt))
    
    res.append(res_image_id)
    
    #============================================================
    
    bbox_idx = 0
    

    if bbox_texts["N"] == [] :
        print("N x")
        print()
        continue
    for bbox_text in bbox_texts["N"]:

        temp = res[int(image_id)-1][bbox_idx].reshape(cnt)
        indices = np.argsort(temp)
        hit_step = 1


        if temp[0] == 0:
            # print(f"{bbox_text} is not in the vocabulary.",bbox_idx)
            bbox_idx+=1
            continue
        # N이 정해짐
        # Y를 정할 차례

        if bbox_texts["Y"] == []:
            hit_step += Top_Hit
            break
            
        for bbox_text_Y in bbox_texts["Y"]:
            for idx in reversed(indices[-Top_Hit:]):
                if word_list[idx] == bbox_text_Y: 
                    break 
                hit_step +=1
  
            # if break_point == True:
            #     break_point = False
            #     break
            if hit_step > Top_Hit: print(f"Hit fails when '{bbox_text}'")
            else: 
                print(f"When '{bbox_text}' hit form",hit_step) 
                break
        # try_count+=1
        
        bbox_idx+=1
    try_count += len(bbox_texts["Y"])
    
    #============================================================
    print("['damaged':'Y'] Bbox Texts:", bbox_texts["Y"])
    print()
print(try_count)



# Y먼저 N나중

In [ ]:
Top_Hit = 50

model = Word2Vec.load("word2vec.model")


json_file_path = "roadsign_data_info_damaged_sample_0928_수정.json"


with open(json_file_path, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)


annotations = data.get("annotations", [])


bbox_texts_by_image_id = {}


for annotation in annotations:
    damaged_status = annotation.get("damaged")
    bbox_text = annotation.get("bbox_text")
    image_id = annotation.get("image_id")
    

    if image_id not in bbox_texts_by_image_id:
        bbox_texts_by_image_id[image_id] = {"N": [], "Y": []}
    
    if damaged_status == "N":
        bbox_texts_by_image_id[image_id]["N"].append(bbox_text)
    elif damaged_status == "Y":
        bbox_texts_by_image_id[image_id]["Y"].append(bbox_text)


res = []
bbox_N_idx = 0
try_count = 0
hit_count = 0
for image_id, bbox_texts in bbox_texts_by_image_id.items():
    
    print("Image ID:", image_id)
    
    res_image_id = []

    print("['damaged':'N'] Bbox Texts:")
    for bbox_text in bbox_texts["N"]:
        if bbox_text in model.wv:
            vector = model.wv[bbox_text]
            reshape_vector = vector.reshape(1, Vector_size)
            res_image_id.append(cosine_similarity(reshape_vector,all_vectors))
            # print(res)
            # print(f"{bbox_text}: {vector}")
        
        else:
            print(f"{bbox_text} is not in the vocabulary.")
            res_image_id.append(np.zeros(cnt))
    
    res.append(res_image_id)
    
    #============================================================
    
    
    
    # if bbox_texts["Y"] == []:
    #     hit_step += Top_Hit
            
            
    for bbox_text_Y in bbox_texts["Y"]:
        # Y이 정해짐
        # N를 정할 차례

        if bbox_texts["N"] == [] :
            print("N x")
            print()
            break

        bbox_idx = 0
        for bbox_text in bbox_texts["N"]:
            temp = res[int(image_id)-1][bbox_idx].reshape(cnt)
            indices = np.argsort(temp)
            if temp[0] == 0:
                # print(f"{bbox_text} is not in the vocabulary.",bbox_idx)
                bbox_idx+=1
                continue
            hit_step = 1
            for idx in reversed(indices[-Top_Hit:]):
                if word_list[idx] == bbox_text_Y: 
                    break 
                hit_step +=1
            bbox_idx+=1

            if hit_step > Top_Hit: print(f"Hit fails when '{bbox_text}'")
            else: 
                hit_count +=1
                print(f"When '{bbox_text}' hit from",hit_step) 
                break
        try_count+=1

    #============================================================
    print("['damaged':'Y'] Bbox Texts:", bbox_texts["Y"])
    print()

print(try_count)
print(hit_count/try_count)



In [ ]:

temp = res[7][0].reshape(cnt)
#print(temp)
#temp = res[image_id][bbox_idx].reshape(cnt)
indices = np.argsort(temp)
print(indices)

for idx in indices[-12:]:
    print(word_list[idx])

In [ ]:
Top_Hit = 100

model = Word2Vec.load("word2vec.model")


json_file_path = "roadsign_data_info_damaged_sample_0928_수정.json"

with open(json_file_path, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)


annotations = data.get("annotations", [])


bbox_texts_by_image_id = {}


for annotation in annotations:
    damaged_status = annotation.get("damaged")
    bbox_text = annotation.get("bbox_text")
    image_id = annotation.get("image_id")
    

    if image_id not in bbox_texts_by_image_id:
        bbox_texts_by_image_id[image_id] = {"N": [], "Y": []}
    
    if damaged_status == "N":
        bbox_texts_by_image_id[image_id]["N"].append(bbox_text)
    elif damaged_status == "Y":
        bbox_texts_by_image_id[image_id]["Y"].append(bbox_text)


res = []
bbox_N_idx = 0
try_count = 0
hit_count = 0
for image_id, bbox_texts in bbox_texts_by_image_id.items():
    
    print("Image ID:", image_id)
    
    res_image_id = []

    for bbox_text in bbox_texts["N"]:
        if bbox_text in model.wv:
            vector = model.wv[bbox_text]
            reshape_vector = vector.reshape(1, Vector_size)
            res_image_id.append(cosine_similarity(reshape_vector,all_vectors))
            # print(res)
            # print(f"{bbox_text}: {vector}")
        
        else:
            # print(f"{bbox_text} is not in the vocabulary.")
            res_image_id.append(np.zeros(cnt))
    
    res.append(res_image_id)
    
    #============================================================
    
    
    
    # if bbox_texts["Y"] == []:
    #     hit_step += Top_Hit
            
            
    for bbox_text_Y in bbox_texts["Y"]:
        # Y이 정해짐
        # N를 정할 차례

        if bbox_texts["N"] == [] :
            # print("N x")
            # print()
            break

        bbox_idx = 0
        for bbox_text in bbox_texts["N"]:
            temp = res[int(image_id)-1][bbox_idx].reshape(cnt)
            indices = np.argsort(temp)
            if temp[0] == 0:
                # print(f"{bbox_text} is not in the vocabulary.",bbox_idx)
                bbox_idx+=1
                continue
            hit_step = 1
            print(f"{bbox_text} : ",end='')
            for idx in reversed(indices[-Top_Hit:]):
                print(word_list[idx], ' ',end ='')
            bbox_idx+=1
            print()


In [ ]:
Top_Hit = 100

model = Word2Vec.load("word2vec.model")


json_file_path = "roadsign_data_info_damaged_sample_0928_수정.json"

with open(json_file_path, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)


annotations = data.get("annotations", [])


bbox_texts_by_image_id = {}


for annotation in annotations:
    damaged_status = annotation.get("damaged")
    bbox_text = annotation.get("bbox_text")
    image_id = annotation.get("image_id")
    

    if image_id not in bbox_texts_by_image_id:
        bbox_texts_by_image_id[image_id] = {"N": [], "Y": []}
    
    if damaged_status == "N":
        bbox_texts_by_image_id[image_id]["N"].append(bbox_text)
    elif damaged_status == "Y":
        bbox_texts_by_image_id[image_id]["Y"].append(bbox_text)


res = []
bbox_N_idx = 0
try_count = 0
hit_count = 0

for image_id, bbox_texts in bbox_texts_by_image_id.items():
    
    print("Image ID:", image_id)
    
    res_image_id = []

    for bbox_text in bbox_texts["N"]:
        if bbox_text in model.wv:
            vector = model.wv[bbox_text]
            reshape_vector = vector.reshape(1, Vector_size)
            res_image_id.append(cosine_similarity(reshape_vector,all_vectors))
            # print(res)
            # print(f"{bbox_text}: {vector}")
        
        else:
            # print(f"{bbox_text} is not in the vocabulary.")
            res_image_id.append(np.zeros(cnt))
    
    res.append(res_image_id)
    
    #============================================================
    
    
    
    # if bbox_texts["Y"] == []:
    #     hit_step += Top_Hit
            
            
    for bbox_text_Y in bbox_texts["Y"]:
        # Y이 정해짐
        # N를 정할 차례

        if bbox_texts["N"] == [] :
            # print("N x")
            # print()
            break

        bbox_idx = 0
        for bbox_text in bbox_texts["N"]:
            temp = res[int(image_id)-1][bbox_idx].reshape(cnt)
            indices = np.argsort(temp)
            res_Array = res[int(image_id)-1][bbox_idx].reshape(cnt)
            if temp[0] == 0:
                # print(f"{bbox_text} is not in the vocabulary.",bbox_idx)
                bbox_idx+=1
                continue
            hit_step = 1
            print(f"{bbox_text} : ",end='')
            for idx in reversed(indices[-Top_Hit:]):
                print("<",word_list[idx], res_Array[idx],">",' ',end ='')
            bbox_idx+=1
            print()


In [ ]:
Top_Hit = 100

model = Word2Vec.load("word2vec.model")


json_file_path = "roadsign_data_info_damaged_sample_0928_수정.json"

with open(json_file_path, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)


annotations = data.get("annotations", [])


bbox_texts_by_image_id = {}


for annotation in annotations:
    damaged_status = annotation.get("damaged")
    bbox_text = annotation.get("bbox_text")
    image_id = annotation.get("image_id")
    

    if image_id not in bbox_texts_by_image_id:
        bbox_texts_by_image_id[image_id] = {"N": [], "Y": []}
    
    if damaged_status == "N":
        bbox_texts_by_image_id[image_id]["N"].append(bbox_text)
    elif damaged_status == "Y":
        bbox_texts_by_image_id[image_id]["Y"].append(bbox_text)


res = []
bbox_N_idx = 0
try_count = 0
hit_count = 0
for image_id, bbox_texts in bbox_texts_by_image_id.items():
    
    print("Image ID:", image_id)
    
    res_image_id = []

    for bbox_text in bbox_texts["N"]:
        if bbox_text in model.wv:
            vector = model.wv[bbox_text]
            reshape_vector = vector.reshape(1, Vector_size)
            res_image_id.append(cosine_similarity(reshape_vector,all_vectors))
            # print(res)
            print(f"{bbox_text}: {vector}")
        
        else:
            # print(f"{bbox_text} is not in the vocabulary.")
            res_image_id.append(np.zeros(cnt))
    
    res.append(res_image_id)

In [ ]:
Top_Hit = 100
model = Word2Vec.load("word2vec.model")

json_file_path = "roadsign_data_info_damaged_sample_0530_수정.json"

with open(json_file_path, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

annotations = data.get("annotations", [])

bbox_texts_by_image_id = {}

for annotation in annotations:
    damaged_status = annotation.get("damaged")
    bbox_text = annotation.get("bbox_text")
    image_id = annotation.get("image_id")
    
 
    if image_id not in bbox_texts_by_image_id:
        bbox_texts_by_image_id[image_id] = {"N": [], "Y": []}
    
    if damaged_status == "N":
        bbox_texts_by_image_id[image_id]["N"].append(bbox_text)
    elif damaged_status == "Y":
        bbox_texts_by_image_id[image_id]["Y"].append(bbox_text)


for Top_Hit in range(1,101):
    res = []
    bbox_N_idx = 0
    try_count = 0
    hit_count = 0
    for image_id, bbox_texts in bbox_texts_by_image_id.items():
        
        # print("Image ID:", image_id)
        
        res_image_id = []
        
        # print("['damaged':'N'] Bbox Texts:")
        for bbox_text in bbox_texts["N"]:
            if bbox_text in model.wv:
                vector = model.wv[bbox_text]
                reshape_vector = vector.reshape(1, Vector_size)
                res_image_id.append(cosine_similarity(reshape_vector,all_vectors))
                # print(res)
                # print(f"{bbox_text}: {vector}")
            
            else:
                # print(f"{bbox_text} is not in the vocabulary.")
                res_image_id.append(np.zeros(cnt))
        
        res.append(res_image_id)
        
        #============================================================
        
        
        
        # if bbox_texts["Y"] == []:
        #     hit_step += Top_Hit
                
                
        for bbox_text_Y in bbox_texts["Y"]:
            # Y이 정해짐
            # N를 정할 차례

            if bbox_texts["N"] == [] :
                break

            bbox_idx = 0
            for bbox_text in bbox_texts["N"]:
                temp = res[int(image_id)-1][bbox_idx].reshape(cnt)
                indices = np.argsort(temp)
                if temp[0] == 0:
                    # print(f"{bbox_text} is not in the vocabulary.",bbox_idx)
                    bbox_idx+=1
                    continue
                hit_step = 1
                for idx in reversed(indices[-Top_Hit:]):
                    if word_list[idx] == bbox_text_Y: 
                        break 
                    hit_step +=1
                bbox_idx+=1

                if hit_step > Top_Hit: pass # print(f"Hit fails when '{bbox_text}'")
                else: 
                    hit_count +=1
                    # print(f"When '{bbox_text}' hit from",hit_step) 
                    break
            try_count+=1

        #============================================================
        # print("['damaged':'Y'] Bbox Texts:", bbox_texts["Y"])
        # print()
    # print(try_count)
    print("Top_hit : " ,Top_Hit , " hit rate : " ,hit_count/try_count)

